<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_reality_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_reality_ai.py

End-to-end pipeline for RealityAI:
1. Synthetic dataset of 6 “reality” inputs → 3 targets
2. Float32 normalization
3. MLP with LayerNorm, Dropout & ReLU
4. Physics-informed residual enforcing toy physical-laws
5. MC-Dropout for uncertainty quantification
6. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, NaN checks, early stopping
7. Checkpointing and safe loading
8. Visualizations: loss curves, scatter plots, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Reality Restructuring Dataset
# ------------------------------------------------------------------------------
class RealityDataset(Dataset):
    def __init__(self, n_samples=5000, seed=0):
        np.random.seed(seed)
        # Features:
        # UC: universal constants factor ∈ [0.5, 2.0]
        # ST: space-time topology factor ∈ [0.5, 1.5]
        # DS: dimensional structure index ∈ [1, 10]
        # GC: gravitational coupling ∈ [0.1, 2.0]
        # EC: electromagnetic constant ∈ [0.1, 2.0]
        # DE: dark energy factor ∈ [1e-12, 1e-10]
        UC = np.random.uniform(0.5, 2.0,   (n_samples,1))
        ST = np.random.uniform(0.5, 1.5,   (n_samples,1))
        DS = np.random.uniform(1.0, 10.0,  (n_samples,1))
        GC = np.random.uniform(0.1, 2.0,   (n_samples,1))
        EC = np.random.uniform(0.1, 2.0,   (n_samples,1))
        DE = np.random.uniform(1e-12,1e-10,(n_samples,1))

        X_raw = np.hstack([UC, ST, DS, GC, EC, DE]).astype(np.float64)

        # Toy physical-law targets:
        # DE1: dimensional expansion ∝ DS * ST / (UC + eps)
        # DM: law modifications ∝ GC * EC / (DE + eps)
        # ECN: energy conservation ∝ UC * DS / (GC + eps)
        eps = 1e-12
        DE1 = DS * ST / (UC + eps)
        DM  = GC * EC / (DE + eps)
        ECN = UC * DS / (GC + eps)

        Y_raw = np.hstack([DE1, DM, ECN]).astype(np.float64)
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # Cast to float32
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. RealityAI Model Definition
# ------------------------------------------------------------------------------
class RealityAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        if isinstance(hidden_dims, int):
            hidden_dims = (hidden_dims,)
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.LayerNorm(h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual & Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    # Denormalize
    X_den = X * stats['X_std'] + stats['X_mean']
    UC, ST, DS, GC, EC, DE = X_den.t()
    eps = 1e-12

    DE1_t = DS * ST / (UC + eps)
    DM_t  = GC * EC / (DE + eps)
    ECN_t = UC * DS / (GC + eps)

    Yt = torch.stack([DE1_t, DM_t, ECN_t], dim=1)
    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    preds = []
    with torch.no_grad():
        for _ in range(T):
            preds.append(model(X))
    arr = torch.stack(preds, dim=0)
    return arr.mean(0), arr.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop with NaN Safety & Checkpointing
# ------------------------------------------------------------------------------
def train(model, tr_loader, va_loader, stats, device,
          lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5
    )

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for ep in range(1, epochs+1):
        # Training
        model.train()
        run = 0.0
        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, _, _ = total_loss(pred, yb, xb, stats, lam)
            if torch.isnan(loss):
                print(f"NaN loss at epoch {ep}, aborting.")
                return history
            optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            run += loss.item() * xb.size(0)
        tr_loss = run / len(tr_loader.dataset)

        # Validation
        model.eval()
        run = 0.0
        with torch.no_grad():
            for xb, yb in va_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                run += loss.item() * xb.size(0)
        va_loss = run / len(va_loader.dataset)

        scheduler.step(va_loss)
        history['train'].append(tr_loss)
        history['val'].append(va_loss)
        print(f"Epoch {ep:03d} | Train {tr_loss:.4e} | Val {va_loss:.4e}")

        # Checkpoint & early stop
        if va_loss < best_val - 1e-8:
            best_val, wait = va_loss, 0
            torch.save(model.state_dict(), "best_reality_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {ep}")
                break

    if os.path.exists("best_reality_ai.pth"):
        model.load_state_dict(torch.load("best_reality_ai.pth",
                                         map_location=device))
    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_history(history):
    plt.figure()
    plt.plot(history['train'], label='Train')
    plt.plot(history['val'],   label='Val')
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

def plot_scatter(y_true, y_pred, title):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M], [m, M], 'r--')
    plt.title(title)
    plt.xlabel("True")
    plt.ylabel("Predicted")
    plt.show()

def plot_uncertainty_heatmap(model, stats, device):
    grid = 100
    ST_space = np.linspace(0.5, 1.5, grid, dtype=np.float32)
    DS_space = np.linspace(1.0, 10.0, grid, dtype=np.float32)
    G1, G2 = np.meshgrid(ST_space, DS_space)
    pts = G1.size

    Xg = torch.zeros((pts, 6), device=device)
    for i in (0,3,4,5):  # UC, GC, EC, DE = mean
        Xg[:,i] = stats['X_mean'][i]
    Xg[:,1] = torch.from_numpy(G1.ravel()).to(device)  # ST
    Xg[:,2] = torch.from_numpy(G2.ravel()).to(device)  # DS

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(G1, G2, U, cmap='magma', shading='auto')
    plt.colorbar(label='Std Dimensional Expansion')
    plt.xlabel('Space-Time Topology')
    plt.ylabel('Dimensional Structure')
    plt.title('Uncertainty Heatmap: Dimensional Expansion')
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = RealityDataset(n_samples=5000, seed=0)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }

    n_val = int(0.2 * len(ds))
    tr_ds, va_ds = random_split(ds, [len(ds)-n_val, n_val])
    tr_loader = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_loader = DataLoader(va_ds, batch_size=256, shuffle=False)

    model = RealityAI(input_dim=6, hidden_dims=32, output_dim=3, p_drop=0.1).to(device)
    history = train(model, tr_loader, va_loader, stats, device,
                    lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10)

    plot_history(history)

    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp_norm = model(X_all).cpu().numpy()
    Yp = Yp_norm * ds.Y_std + ds.Y_mean
    Yt = ds.Y * ds.Y_std + ds.Y_mean

    names = ["Dimensional Expansion","Law Modifications","Energy Conservation"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    plot_uncertainty_heatmap(model, stats, device)